Analyzing G25 corrdinates of ancient DNA samples using graphs

In [1]:
require(rio)
require(tidyverse)
require(cowplot)
require(dbscan)
require(igraph)
require(tidygraph)
require(ggraph)
require(pals)

Loading required package: cowplot


Attaching package: ‘cowplot’


The following object is masked from ‘package:lubridate’:

    stamp


Loading required package: dbscan


Attaching package: ‘dbscan’


The following object is masked from ‘package:stats’:

    as.dendrogram


Loading required package: igraph


Attaching package: ‘igraph’


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following objects are masked from ‘package:lubridate’:

    %--%, union


The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


The following objects are masked from ‘package:purrr’:

    compose, simplify


The following object is masked from ‘package:tidyr’:

    crossing


The following object is masked from ‘package:tibble’:

    as_data_frame


The following object is masked from ‘package:base’:

    union


Loading required package: tidygraph


Attaching package: ‘tidygraph’


The following object is masked from ‘package:igr

Load the data for modern and ancient G25 population averages

In [ ]:
avgs = as_tibble(import("~/Documents/Ancestry/Genetics/G25/Data/TXT/Global25_PCA_pop_averages_scaled.txt"))
#avgs %>% sample_n(10)
modavgs = as_tibble(import("~/Documents/Ancestry/Genetics/G25/Data/TXT/Global25_PCA_modern_pop_averages_scaled.txt"))
#modavgs %>% sample_n(10)

Optionally, filter the dataset down

In [ ]:
str_extract(avgs$V1,"^\\w+?(?=_)") %>% unique(sort = TRUE)

In [ ]:
str_extract(modavgs$V1,"^\\w+?$") %>% unique(sort = TRUE)

In [ ]:
str_extract(modavgs$V1,"^\\w+?$") %>% unique(sort = TRUE) %>% as.character() %>% paste0(collapse = "|")

In [ ]:
avgs = avgs %>% dplyr::filter(!str_detect(V1,"^(ARG|AUS|BHS|BLZ|BRA|BWA|CAN|CHL|CHN|CMR|COG|CUW|DOM|ETH|GUM|HTI|IDN|IND|JPN|KAZ|KEN|KGZ|KOR|LAO|MEX|MNG|MWI|MYS)"))
avgs = avgs %>% dplyr::filter(!str_detect(V1,"^(NPL|PAK|PAN|PER|PYF|Saka|Sarg|SDN|THA|TJK|TKM|TON|TWN|TZA|UGA|USA|UZB|VEN|VNM|VUT|ZAF)"))
avgs = avgs %>% dplyr::filter(!str_detect(V1,"_o$"))
avgs = avgs %>% dplyr::filter(!str_detect(V1,"low"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Abazin|Abkhasian|Adygei|Aeta|Afrikaner|Agta|Ahiska|Akha|Akhvakh|Alawite|Alevi|Algerian|Altaian|Amerindian|Ami|Andian|Arain)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Armenian_[AGHP]|Arora|Assyrian|Asur|Atayal|Australian|Awan|Aymara|Azerbaijani_R|Basque_[ABGLR]|Batak|Bedzan|Bengali|Berber_[AM])"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Arora|Assyrian|Asur|Atayal|Australian|Awan|Aymara|Azerbaijani_R|Batak|Bedzan|Bengali|Bagvalin|Bahun|Baiku|Balti|Baniya|Bantu)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Besermyan|Bhumi|Biaka|Birhor|Bitonga|Blang|Bolivian|Bonan|Bonda|Brahmin|Brahui|Bulala|Bunt|Burmese|Burusho|Buryat|Cachi|Cambodian)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Cameroon|Chama|Chan|Chen|Chipewyan|Chopi|Chukchi|Chuvash|Circassian|Cochin|Colla|Cree|Dai|Damai|Darginian|Datog|Daur|Dharkar|Dinka)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Dolgan|Dong|Dungan|Dusadh|Dusun|Elmolo|Eritrean|Esan_Nigeria|Eskimo|Ethiopian|Even|Evenk|Ezhava|Ezid)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(French_Au|French_B[ei]|French_[COP]|Fulani|Gadaba|Gagauz|Gambian|Ganguela|Gelao|Georgian_[GIJKLMRST]|Gond|Greek_[ADEIKLMNSTW])"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Gujar|Gurung|Hadza|Hakkipikki|Han|Hawaiian|Hazara|Hezhen|Hinukh|Hmong|Ho|Htin_Mal|Hui|Hunzib|Igbo|Igorot|Indonesian|Iranian_[BJLMP])"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Iraq|Irula|Italian_[ABEFJLM]|Itelmen|Jamatia|Japanese|Jarawa|Jat|Jehai|Ju_hoan_North|Juang|Kaba|Kadar|Kaitag|Kalash|Kalmyk|Kamboj)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Irula|Itelmen|Jamatia|Japanese|Jarawa|Jat|Jehai|Ju_hoan_North|Juang|Kaba|Kadar|Kaitag|Kalash|Kalmyk|Kamboj)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Kamma|Kanjar|Karachay|Karaite_Egypt|Karakalpak|Karata|Karen_Sgaw|Karitiana|Kashmiri|Kazakh|Ket|Khakass|Khamnegan|Khanty|Khatri)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Khmer|Kho|Kikuyu|Kinh_Vietnam|Kirghiz|Knanaya|Kohistani|Koinanbe|Kol|Komi|Kongo|Konkani|Korean|Korwa|Koryak|Kosipe)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Kshatriya|Kubachinian|Kumyk|Kurdish|Kurichiya|Kurumba|Kusunda|Kuy_Suay|Lahu|Laka|Lao|Lawa|Lebbo|Lemande|Lezgin|Li|Libyan)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Lithuanian_[RP]|Luhya_Kenya|Luo|Luzon|Macedonian|Mada|Madagascar|Madiga|Magar|Makhuwa|Makrani|Mala|Malay|Manchu|Mandenka|Maniq)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Maniyani_Kerala|Mansi|Manyika|Maonan|Maori|Maratha|Mari|Masai|Mayan|Mbuti|Mende_Sierra_Leone|Miao|Mix|Mlabri|Mogush|Mon|Moroccan_)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Mountain_|Mozabite|Mulam|Murut|Mwani|Nahua|Nair|Nanai|Nasoi|Nasrani|Naxi|Ndau|Negidal|Nenets|Nepali|Newar|Nganassan|Ngumba|Nivkh|Nogai)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(North|Nyah_Kur|Nyan|Ogiek|Onge|Orcadian|Oroqen|Palestinian_|Pallan|Pamiri|Paniya|Papuan|Parsi|Pashtun|Pathan_Bhopal|Piapoco|Pima)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Piramalai_Kallar|Poduval_Kerala|Polish_|Pul|Pumi|Punj|Qiang|Qing|Quechua|Rai|Raj|Ratlub|Reddy|Relli|Rendille|Riang|Rohingya)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Roma_|Romaniote|Ronga|Ror|Rumelia_East|Russian_[BKRSTVY]|Saharawi|Sakha|Sakilli|Salar|Saliya_Kerala|Samaritan|Sandawe|Santhal)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Satnami_Chhattisgarh|Saudi[AB]|Selkup|Sena|Sengwer|She|Shor|Sindhi|Somali|Spanish_[ABCEGLPS]|Sri_Lankan|Sudanese|Surui)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Syed|Tabasaran|Tai_Lue|Tajik|Talysh_Azerbaijan|Tamang|Tamil_Sri_Lanka|Tarkhan_Muslim|Tat|Telugu|Thai|Tharu|Thiyya|Tibetan)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Tikar_South|Tindal|Tlingit|Todzin|Tripuri|Tsez|Tswa|Turkish_[ABDEGKNS]|Turkmen|Tuvinian|Tyagi|Udi|Udmurt|Ulchi|Umbundu)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Uttar|Uygur|Uzbek|Vaniya_Kerala|Velama|Vellalar|Vishwakarma_Kerala|Vizayan|Wa|Welsh|Wichi|Xibo|Yadav_Telugu|Yakut|Yao|Yemenite)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Yi|Yoruba|Yugur|Yukagir|Yukpa|Yuku|Zapotec|Zhuang)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"_o$"))

In [ ]:
avgs$V1 %>% unique(sort = TRUE)
modavgs$V1 %>% unique(sort = TRUE)

In [ ]:
myavgs = avgs %>% select(-V1)
row.names(myavgs) = avgs$V1
mymodavgs = modavgs %>% select(-V1)
row.names(mymodavgs) = modavgs$V1

Combine all data

In [ ]:
allavgs = bind_rows(myavgs,mymodavgs)

Find nearest neighbors for each row

In [ ]:
mynn = kNN(allavgs,k = nrow(allavgs)-1)
mynn

In [ ]:
mynn_d = rownames_to_column(as.data.frame(round(mynn$dist,2))) %>% as_tibble() %>% mutate(`0` = 0,.after = rowname)
mynn = rownames_to_column(as.data.frame(mynn$id)) %>% as_tibble() %>% mutate(`0` = row_number(),.after = rowname)

In [ ]:
mynn %>% sample_n(5)
mynn_d %>% sample_n(5)

Pivot and clean up the data

In [ ]:
mynames = mynn %>% select(rowname) %>% mutate(val = row_number())

In [ ]:
mynn = mynn %>% pivot_longer(cols = matches("^\\d"),names_to = "rank",values_to = "val") %>% inner_join(mynames,by = "val") %>% select(c(1,4,2)) # nolint: line_length_linter.
mynn_d = mynn_d %>% pivot_longer(cols = matches("^\\d"),names_to = "rank",values_to = "dist")

In [ ]:
mynn %>% sample_n(5)
mynn_d %>% sample_n(5)
#mynn_d %>% count(rank)

In [ ]:
colnames(mynn) = c("target","population","rank")
colnames(mynn_d) = c("target","rank","distance")
mynn = mynn %>% inner_join(mynn_d) %>% dplyr::filter(rank %in% 1:5)
mynn %>% sample_n(10)

In [ ]:
write_csv(mynn,"../Genetics/G25/Data/CSV/top_10_dist_all.csv")

In [ ]:
mynn %>% dplyr::filter(str_detect(target,"^Ukrainian.+[^o]$"))

In [ ]:
myg = graph_from_data_frame(mynn)

In [ ]:
myg

In [ ]:
myg = as_tbl_graph(myg)
myg

In [ ]:
mypf = function (mygr,wdth = 12,hgth = 12,ppi = 400,bgrd = "gray20") {
    myg = mygr %>% mutate(c_auth = centrality_authority(weights = 1/distance)) 
    print(myg %>% select(name,c_auth) %>% as.data.frame() %>% arrange(desc(c_auth)))
    myp = myg %>% ggraph(layout = 'stress',circular = TRUE) 
    options(repr.plot.width = wdth,repr.plot.height = hgth,repr.plot.res = ppi,repr.plot.bg = bgrd)
    myp + geom_edge_fan(aes(color = distance,alpha = after_stat(index),start_cap = label_rect(node1.name),end_cap = label_rect(node2.name)),strength = 1.25,edge_width = 0.15) + 
        geom_node_label(aes(label = name,color = c_auth),label.size = 0,size = 1,alpha = 0.6,fill = "gray15") + #,repel = TRUE,max.overlaps = 10) + 
        scale_edge_alpha('Edge direction') + 
        scale_edge_color_gradient2("distance",low = "cyan",mid = "snow",high = "orange") + 
        scale_color_gradient2("centrality",low = "cyan",mid = "snow",high = "orange") + 
        theme_void() + 
        theme(plot.background = element_rect("gray20"),
              legend.title = element_text(color = "snow",size = 5),
              legend.text = element_text(color = "snow",size = 4),
              legend.position = "bottom") + #,legend.key.size = unit(4,'pt')) +
        guides(edge_alpha = guide_edge_direction()) + 
        coord_flip()
    ggsave("~/Desktop/plot.pdf",width = wdth,height = hgth,dpi = ppi)
    }

In [ ]:
mynames$rowname

In [ ]:
myg %>% convert(to_local_neighborhood,node = which(str_detect(.N()$name,"Gnezdovo")),order = 5,mode = "all") %>% mypf() # nolint: line_length_linter.